# 슈퍼바이저 에이전트(Supervisor Agent) 구축하기

**슈퍼바이저 패턴(Supervisor Pattern)** 은 **중앙 감독 에이전트(Supervisor Agent)** 가 여러 **전문 하위 에이전트(Worker Agents)** 를 조율하는 **멀티 에이전트 아키텍처**입니다.

이 접근 방식은 **서로 다른 전문 지식이 필요한 작업**을 수행할 때 특히 효과적입니다.
즉, 하나의 에이전트가 모든 도구 선택과 작업을 처리하는 대신, 각 분야별 **전문 에이전트(specialized agent)** 를 만들고,
**슈퍼바이저가 전체 워크플로우를 이해하고 관리**하도록 하는 구조입니다.

---

이 노트북에서는 이러한 장점을 보여주는 **워크플로우 기반의 개인 비서 시스템(personal assistant system)** 을 구축합니다.

역할이 근본적으로 다른 두 개의 전문가 에이전트를 조율합니다.

1. **캘린더 에이전트 (Calendar Agent)** : 일정 관리, 가용 시간 확인, 이벤트 생성 및 관리 담당  
2. **이메일 에이전트 (Email Agent)** : 커뮤니케이션 관리, 메일 초안 작성, 알림 발송 담당

## 왜 슈퍼바이저를 사용하는가?

멀티 에이전트 아키텍처는
**각 작업자(Worker)** 에이전트가 **자신만의 도구 세트와 프롬프트(지침)** 를 가지도록 역할을 분리(partition)할 수 있게 해줍니다.

예를 들어, 하나의 에이전트가 **캘린더 API** 와 **이메일 API** 모두에 직접 접근한다고 가정해봅시다.
이 경우 에이전트는 다음과 같은 문제를 동시에 해결해야 합니다.

* 여러 개의 **유사한 도구들** 중 어떤 것을 사용할지 결정해야 하고,
* 각 API의 **정확한 입력 형식(format)** 을 이해해야 하며,
* 서로 다른 **도메인(일정 관리, 이메일 통신 등)** 을 동시에 처리해야 합니다.

이처럼 복잡한 구조는 **성능 저하(performance degradation)** 를 초래할 수 있습니다.
이럴 때, 관련된 도구들과 프롬프트를 **논리적 그룹(logical group)** 으로 분리하는 것이 도움이 됩니다.
이 방식은 **유지보수 및 반복적 개선(iterative improvement)** 을 용이하게 하며,
각 하위 에이전트를 보다 **전문화된 형태로 최적화**할 수 있게 해줍니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

## 1. 도구 정의하기 (Define Tools)

먼저 **구조화된 입력(Structured Input)** 이 필요한 도구들을 정의합니다.
실제 애플리케이션에서는 이러한 도구들이 예를 들어 **Google Calendar**, **Notion** 등의 **실제 API** 를 호출하게 됩니다.

In [ ]:
def create_calendar_event(
    # 실제로는 Google Calendar API, Outlook API 등을 호출합니다.
def send_email(
    # 실제로는 SendGrid, Gmail API 등을 호출합니다.
def get_available_time_slots(
    # 실제로는 Google Calendar 등의 캘린더 API를 조회합니다.

## 2. 전문 하위 에이전트(Sub-Agent) 생성하기

다음 단계에서는 **각 도메인별 작업을 담당하는 전문 하위 에이전트(Sub-Agent)** 를 만듭니다.

**캘린더 에이전트**는 사용자의 **자연어 일정 요청(natural language scheduling requests)** 을 정확한 **API 호출로 변환**하는 역할을 합니다.
즉, 사용자가 “내일 오후 2시에 회의 잡아줘”라고 말하면 이를 이해하고, ISO 형식의 API 호출로 변환하여 실제 일정 등록 작업을 수행합니다.

- `calendar_agent` test

In [ ]:
# 캘린더 에이전트를 스트리밍 모드로 실행
    # 각 단계(step)의 업데이트 내용을 순회
        # 메시지가 존재할 경우 메시지들을 출력

-------------------
**이메일 에이전트**는 메시지 작성 및 전송을 담당합니다.  
사용자가 “팀에게 회의 일정 공유 이메일 보내줘”라고 말하면, 이메일 에이전트는 수신자 목록을 추출하고, 자연스러운 제목과 내용을 자동으로 생성하여
**이메일 발송 프로세스 전체를 관리**합니다.

- email_agent test

In [ ]:
# 이메일 에이전트를 스트리밍 모드로 실행
    # 각 단계(step)의 업데이트 내용을 순회
        # 메시지가 존재할 경우 메시지들을 출력

## 3. 하위 에이전트를 도구로 감싸기 

이제 각 **하위 에이전트(Sub-Agent)** 를 **슈퍼바이저(Supervisor)** 가 호출할 수 있는 **도구(Tool)** 형태로 감쌉니다.  
이 단계가 바로 **계층형 시스템(layered system)** 을 만드는 핵심 아키텍처 단계입니다.  

슈퍼바이저는 더 이상 세부적인 하위 함수(`create_calendar_event` 등)를 직접 호출하지 않습니다.  
대신, **고수준 도구(high-level tools)** — 예를 들어 `"schedule_event"` 또는 `"send_email"` 과 같은 추상화된 명령을 사용할 수 있습니다.

In [ ]:
def schedule_event(request: str) -> str:
def manage_email(request: str) -> str:

도구의 **설명(Description)** 은 슈퍼바이저가 **언제 어떤 도구를 사용할지 결정**하는 데 도움을 줍니다.
따라서 설명은 **명확하고 구체적**이어야 합니다.

또한, 하위 에이전트의 실행 결과 중에서 **최종 응답(final response)** 만 반환하도록 해야 합니다.
슈퍼바이저는 하위 에이전트의 **중간 추론(reasoning)** 이나 **도구 호출 과정(tool calls)** 을 볼 필요가 없기 때문입니다.

---

## 4. 슈퍼바이저 에이전트 생성하기

이제 여러 하위 에이전트를 **조율(orchestrate)** 하는 **슈퍼바이저(Supervisor Agent)** 를 만듭니다.

슈퍼바이저는 다음과 같은 방식으로 작동합니다.

* 하위 에이전트들이 제공하는 **고수준 도구(high-level tools)** 만 인식합니다.
* **도메인 단위(domain-level)** 에서 라우팅 결정을 내리며, 세부적인 API 호출 수준의 작업은 직접 다루지 않습니다.

## 5. Supervisor 사용
여러 도메인 간 조율이 필요한 복합 요청으로 완성된 시스템 테스트

### 예시 1: 단일 도메인 요청 (Simple Single-Domain Request)
- 일정 관리(Calendar) 도메인만 포함 → schedule_event 도구만 사용

In [ ]:
# 슈퍼바이저 에이전트를 스트리밍 모드로 실행
    # 각 단계(step)의 업데이트 내용을 순회
        # 메시지가 존재할 경우 메시지들을 출력

-----------------
### 예시 2: 복합 다중 도메인 요청 (Complex Multi-Domain Request)

- 두 개의 도메인 작업이 결합
    - 일정 관리 (Calendar)
    - 이메일 발송 (Email)

Supervisor가 2개의 도구를 순차적으로 호출해야 함: schedule_event → manage_email

In [ ]:
# 슈퍼바이저 에이전트를 스트리밍 모드로 실행
    # 각 단계(step)의 업데이트 내용을 순회
        # 메시지가 존재할 경우 메시지들을 출력

-------------------
### 아키텍처 이해하기 

이 시스템은 **3계층 구조(three-layer architecture)** 로 이루어져 있습니다.

1. 하위 계층 (Bottom Layer)

* **정확한 형식(exact formats)** 을 요구하는 **고정형 API 도구(rigid API tools)** 로 구성됩니다.
* 예: `create_calendar_event`, `send_email` 등
* 이 계층은 실제 외부 서비스(API)와 직접 상호작용합니다.

2. 중간 계층 (Middle Layer)

* **하위 에이전트(Sub-Agents)** 가 존재하는 계층입니다.
* 사용자의 **자연어 입력(natural language)** 을 받아
  이를 **구조화된 API 호출(Structured API Calls)** 로 변환한 뒤,
  다시 **자연어 응답(natural language confirmations)** 형태로 반환합니다.
* 예: `calendar_agent`, `email_agent`

3. 상위 계층 (Top Layer)

* **슈퍼바이저(Supervisor Agent)** 가 위치한 계층입니다.
* 하위 에이전트들을 조율하며, **고수준 명령(high-level capabilities)** 을 인식하고
  각 도메인별로 요청을 **라우팅(routing)** 합니다.
* 또한 여러 하위 작업의 결과를 **통합(synthesize)** 하여 사용자에게 제공합니다.